In [28]:
import requests
import json
from datetime import datetime
import sys
import csv

# Importer la fonction get_access_token du module auth
from auth import get_access_token

# Constantes et fichiers de configuration
CLIENT_ID = 'vss6aqb3gf2czn5syafmbrzp3'
CLIENT_SECRET = 'VzZCePAxmQ'

In [29]:
# Constantes et fichiers de configuration
API_URL_TEMPLATE = "https://api.lufthansa.com/v1/operations/flightstatus/departures/{airportCode}/{fromDateTime}"
AIRPORT_CODES_FILE = "data/json/airline_codes.json"
JSON_RESULTS_FILE = "data/json/departure_airports.json"
CSV_RESULTS_FILE = "data/csv/departure_airports.csv"
JSON_ERRORS_FILE = "data/json/api_errors.json"
CSV_ERRORS_FILE = "data/csv/api_errors.csv"

def load_airport_codes(file_path):
    """Charge les codes d'aéroports depuis un fichier JSON."""
    with open(file_path, "r") as json_file:
        data = json.load(json_file)
    return data.get('airport_codes', [])

def load_previous_results(file_path):
    """Charge les résultats précédents depuis un fichier JSON, ou retourne une liste vide si le fichier n'existe pas."""
    try:
        with open(file_path, "r") as json_file:
            return json.load(json_file)
    except FileNotFoundError:
        return []

def fetch_flight_status(airport_code, from_datetime, headers):
    """Appelle l'API pour obtenir le statut des vols d'un aéroport donné."""
    url = API_URL_TEMPLATE.format(airportCode=airport_code, fromDateTime=from_datetime)
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {airport_code}: {http_err}")
        return {"error": str(http_err), "status_code": response.status_code, "reason": response.reason}
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred for {airport_code}: {req_err}")
        return {"error": str(req_err)}

In [30]:
def load_existing_json(file_path):
    """Loads existing JSON data from file if available."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except (IOError, json.JSONDecodeError):
        return []

def save_json_to_file(data, file_path, data_type):
    """Appends data to a JSON file."""
    existing_data = load_existing_json(file_path)
    combined_data = existing_data + data
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(combined_data, f, ensure_ascii=False, indent=4)
        print(f"{data_type} ont été enregistrées dans {file_path}")
    except IOError as e:
        print(f"Erreur lors de l'écriture du fichier {file_path}: {e}")

def save_to_csv(data, file_path, data_type):
    """Appends data to a CSV file."""
    keys = data[0].keys()
    try:
        with open(file_path, "a", newline='', encoding="utf-8") as output_file:
            dict_writer = csv.DictWriter(output_file, fieldnames=keys)
            if output_file.tell() == 0:
                dict_writer.writeheader()
            dict_writer.writerows(data)
        print(f"{data_type} ont été mises à jour dans {file_path}")
    except IOError as e:
        print(f"Erreur lors de l'écriture du fichier {file_path}: {e}")

In [32]:
import requests
import json
import csv
from datetime import datetime
from auth import get_access_token

# Constantes et fichiers de configuration
API_URL = "https://api.lufthansa.com/v1/mds-references/aircraft"
JSON_OUTPUT_FILE = "data/json/aircrafts.json"
CSV_OUTPUT_FILE = "data/csv/aircrafts.csv"
OFFSET = 0
LIMIT = 100

def fetch_data(api_url, headers, offset, limit):
    """Fetches data from the given API URL with provided headers."""
    url = f"{api_url}?offset={offset}&limit={limit}"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")
    return None


def main():
    """Main function to execute the script."""
    access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)
    if not access_token:
        print("Échec de l'obtention du token d'accès.")
        return

    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }

    print("Fetching aircraft data...")
    all_aircraft = []
    offset = OFFSET

    while True:
        data = fetch_data(API_URL, headers, offset, LIMIT)
        if data and 'AircraftResource' in data and 'AircraftSummaries' in data['AircraftResource']:
            aircraft_list = data['AircraftResource']['AircraftSummaries']['AircraftSummary']
            if not aircraft_list:
                break
            all_aircraft.extend(aircraft_list)
            offset += LIMIT
        else:
            break

    print(f"Total aircraft retrieved: {len(all_aircraft)}")
    save_json_to_file(all_aircraft, JSON_OUTPUT_FILE, "données des avions")
    save_to_csv(all_aircraft, CSV_OUTPUT_FILE, "données des avions")

if __name__ == "__main__":
    main()

Access Token: fy2jv5p3nderbfwqdcfjwch6
Fetching aircraft data...
HTTP error occurred: 404 Client Error: Not Found for url: https://api.lufthansa.com/v1/mds-references/aircraft?offset=400&limit=100
Total aircraft retrieved: 383
données des avions ont été enregistrées dans data/json/aircrafts.json
données des avions ont été mises à jour dans data/csv/aircrafts.csv
